# CHIPPR

This notebook demonstrates the use of the Cosmological Hierarchical Inference with Probabilistic Photometric Redshifts (CHIPPR) package to estimate the redshift density function.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import timeit
import cProfile, pstats, StringIO

import chippr
from chippr import *

In [ ]:
#help(chippr)

## Simulation

In [ ]:
tru_amps = np.array([0.20, 0.35, 0.55])
tru_means = np.array([0.5, 0.2, 0.75])
tru_sigmas = np.array([0.4, 0.2, 0.1])

tru_nz = gmix(tru_amps, tru_means, tru_sigmas, limits=(0., 1.))

In [ ]:
N = 2**16

tru_zs = tru_nz.sample(N)

plt.hist(tru_zs, bins=50, normed=1)
plt.xlabel(r'true $z$')
plt.ylabel(r'probability')

In [ ]:
params = 'params.txt'
params = sim_utils.ingest(params)
print(params)

In [ ]:
bin_ends = np.array([0., 1.])
weights = np.array([1.])

int_prior = discrete(bin_ends, weights)

In [ ]:
pr = cProfile.Profile()
pr.enable()

posteriors = catalog(params)
output = posteriors.create(tru_zs, int_prior, bins=16)

data = np.exp(output['log_interim_posteriors'])

pr.disable()
s = StringIO.StringIO()
sortby = 'tottime'
ps = pstats.Stats(pr, stream=s).sort_stats(sortby)
ps.print_stats()
print s.getvalue()

In [ ]:
# for datum in data[:10]:
#     plt.plot(posteriors.x_coarse, datum)
# plt.plot(posteriors.x_coarse, int_prior.evaluate(posteriors.x_coarse), color='k')
plt.scatter(tru_zs, posteriors.obs_samps)
plt.xlabel(r'$z_{true}$')
plt.ylabel(r'$z_{obs}$')

In [ ]:
plt.hist(posteriors.obs_samps, bins=100, normed=True, color="k")
plt.plot(posteriors.x_coarse, tru_nz.evaluate(posteriors.x_coarse), "r-")
plt.plot(posteriors.x_coarse, np.sum(data, axis=0) / N, "go")
plt.xlabel("z")

In [ ]:
for n, z in enumerate(data[:10]):
    plt.plot(posteriors.x_coarse, data[n], 'ko')
    plt.plot(posteriors.x_fine, posteriors.obs_lfs[n], 'k-')
    plt.show()

## Inference

In [ ]:
# data_loc = '../../../prob-z/nz/code/tests/sig2/data/logdata.csv'

# with open(data_loc,'rb') as data_file:
#     tuples = (line.split(None) for line in data_file)
#     data = [[float(pair[k]) for k in range(0,len(pair))] for pair in tuples]

In [ ]:
zs = output['bin_ends']
nz_intp = np.exp(output['log_interim_prior'])
z_posts = data

z_difs = zs[1:]-zs[:-1]
z_mids = (zs[1:]+zs[:-1])/2.
n_bins = len(z_mids)

In [ ]:
# prior_sigma = 0.16
# prior_var = np.eye(n_bins)
# for b in range(n_bins):
#     prior_var[b] = 1. * np.exp(-0.5 * (z_mids[b] - z_mids) ** 2 / prior_sigma ** 2)
# l = 1.e-4
# prior_var = prior_var+l*np.identity(n_bins)

prior_var = np.eye(n_bins)
for k in range(n_bins):
    prior_var[k] = 1. * np.exp(-0.5 * (z_mids[k] - z_mids) ** 2 / 0.05 ** 2)

prior_mean = nz_intp
prior = mvn(prior_mean, prior_var)

In [ ]:
nz = log_z_dens(output, prior, truth=tru_nz, vb=True)

In [ ]:
nz_map = nz.calculate_mmap()

In [ ]:
nz_stack = nz.calculate_stack()

In [ ]:
pr = cProfile.Profile()
pr.enable()

nz_mmle = nz.calculate_mmle(nz_stack)

pr.disable()
s = StringIO.StringIO()
sortby = 'tottime'
ps = pstats.Stats(pr, stream=s).sort_stats(sortby)
ps.print_stats()
print s.getvalue()

In [ ]:
nz.plot()

In [ ]:
#nz_mexp = nz.mexp()

In [ ]:
#nz_samps = nz.sample()